<a href="https://colab.research.google.com/github/msaantonova/Accent-Classification_Speech-technologies/blob/main/Accent_recognition_project_SS25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

Repository from the paper

"This repository provides all the necessary tools to perform accent identification from speech recordings with SpeechBrain toolkit! The system uses a model fine-tuned on the CommonAccent dataset in English (21 accents). The provided system can recognize the following 21 accents of English from short speech recordings"

In [4]:
!git clone https://github.com/lgspeech/accent-recog-slt2022

Cloning into 'accent-recog-slt2022'...
remote: Enumerating objects: 527, done.
remote: Counting objects: 100% (204/204), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 527 (delta 182), reused 155 (delta 154), pack-reused 323 (from 1)
Receiving objects: 100% (527/527), 2.64 MiB | 21.25 MiB/s, done.
Resolving deltas: 100% (342/342), done.


In [5]:
%cd /content/accent-recog-slt2022
!python -m pip install -r requirements.txt
# you can ignore any errors for now

/content/accent-recog-slt2022
Ignoring SoundFile: markers 'sys_platform == "win32"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 50.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Ignored the following yanked versions: 2.0.0
ERROR: Ignored the following versions that require a different python version: 1.6.2 Requires-Python >=3.7,<3.10; 1.6.3 Requires-Python >=3.7,<3.10; 1.7.0 Requires-Python >=3.7,<3.10; 1.7.1 Requires-Python >=3.7,<3.10; 1.7.2 Requires-Python >=3.7,<3.11; 1.7.3 Requires-Python >=3.7,<3.11; 1.8.0 Requires-Python >=3.8,<3.11; 1.8.0rc1 Requires-Python >=3.8,<3.11; 1.8.0rc2 Requires-Python >=3.8,<3.11; 1.8.0rc3 Requires-Python >=3.8,<3.11; 1.8.0rc4 Requires-Python >=3.8,<3.11; 1.8.1 Requires-Python >=3.8,<3.11
ERROR: Could not find a versi

In [6]:
!pip install -q datasets==3.0.0

In [7]:
# change working directory:
%cd CommonAccent/

/content/accent-recog-slt2022/CommonAccent


# Download the data (or stream) and create the list of filtered items

You can either use streaming, or save the files locally. you can check at common voice, how big the corpus for each language is.
If you use streaming, it  will take a few hours.

The aim of it is to create lists of all audios that have marked accent in the metadata, so you can choose which ones of these you want for classification.

In [ ]:
#this part descrbes the dataset index and the language
!python download_data_hf.py --language "it" data/cv_11/

common_voice_19_0.py: 100% 8.17k/8.17k [00:00<00:00, 33.2MB/s]
README.md: 100% 4.47k/4.47k [00:00<00:00, 19.0MB/s]
languages.py: 100% 4.00k/4.00k [00:00<00:00, 23.3MB/s]
release_stats.py: 100% 138k/138k [00:00<00:00, 1.81MB/s]
n_shards.json: 100% 16.5k/16.5k [00:00<00:00, 51.0MB/s]
it_train_0.tar: 100% 1.69G/1.69G [00:15<00:00, 112MB/s]
it_train_1.tar: 100% 1.58G/1.58G [00:18<00:00, 83.6MB/s]
it_train_2.tar: 100% 1.38G/1.38G [00:13<00:00, 99.3MB/s]
it_train_3.tar: 100% 1.32G/1.32G [00:13<00:00, 94.7MB/s]
it_train_4.tar: 100% 374M/374M [00:02<00:00, 138MB/s]
it_dev_0.tar: 100% 703M/703M [00:06<00:00, 115MB/s]
it_test_0.tar: 100% 743M/743M [00:08<00:00, 84.5MB/s]
it_other_0.tar: 100% 443M/443M [00:11<00:00, 38.8MB/s]
it_invalidated_0.tar: 100% 875M/875M [00:13<00:00, 65.6MB/s]
train.tsv: 100% 54.7M/54.7M [00:01<00:00, 47.9MB/s]
dev.tsv: 100% 4.65M/4.65M [00:00<00:00, 18.1MB/s]
test.tsv: 100% 4.62M/4.62M [00:00<00:00, 16.5MB/s]
other.tsv: 100% 3.90M/3.90M [00:00<00:00, 21.7MB/s]
invalidat

# Prepare the filtered list to be used in this model --> create csv files in correct form

In [ ]:
!pip install -q speechbrain==0.5.13 \
    transformers==4.25.1 \
    librosa==0.9.2 \
    ipdb

In [ ]:
!python common_accent_prepare.py --language "it" data/cv_11 data/

data/cv_11 ['.mp3'] it
/content/accent-recog-slt2022/CommonAccent/data/cv_11/it/validation.tsv
/content/accent-recog-slt2022/CommonAccent/data/cv_11/it/test.tsv
/content/accent-recog-slt2022/CommonAccent/data/cv_11/it/train.tsv


In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.io.wavfile import write
from datasets import load_dataset

data_path = "/content/accent-recog-slt2022/CommonAccent/data/"
# Load CSV
csv_path_train = data_path + "train.csv"
df_train = pd.read_csv(csv_path_train)
file_list_train = list(set(df_train['wav']))

csv_path_test = data_path + "test.csv"
df_test = pd.read_csv(csv_path_test)
file_list_test = list(set(df_test['wav']))

csv_path_dev = data_path + "dev.csv"
df_dev = pd.read_csv(csv_path_dev)
file_list_dev = list(set(df_dev['wav']))

# Set save directory
save_dir = data_path + "wav_files"
os.makedirs(save_dir, exist_ok=True)

# Load dataset in streaming mode
language = "it"  # or whatever language code you're using
cv_19_train = load_dataset("fsicoli/common_voice_19_0", language, streaming=True, split="train", trust_remote_code=True)
print(next(iter(cv_19_train)))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Reading metadata...: 171388it [00:07, 23662.65it/s]


{'client_id': 'c14f21cacf2b7701ad0bead0dd1b31ec9d3a1557708e446de55e98b4b470cf31072c82543e5ba518c5c187a91868878a4e32727054a3dd94f9df41c9a13d8c62', 'path': 'it_train_0/common_voice_it_17415777.mp3', 'audio': {'path': 'it_train_0/common_voice_it_17415777.mp3', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
       -1.61342832e-05, -1.57291943e-05, -1.16234423e-05]), 'sampling_rate': 48000}, 'sentence': "Il marchese aveva già moglie in quell'epoca?", 'up_votes': 3, 'down_votes': 0, 'age': '', 'gender': '', 'accent': '', 'locale': 'it', 'segment': '', 'variant': ''}


In [ ]:
cv_19_dev = load_dataset("fsicoli/common_voice_19_0", language, streaming=False, split="validation", trust_remote_code=True)
cv_19_test = load_dataset("fsicoli/common_voice_19_0", language, streaming=False, split="test", trust_remote_code=True)

In [ ]:
import pandas as pd
# Count entries per accent
accent_counts = df_train['accent'].value_counts()

# Print the result
print(accent_counts)

# select randomly max 300 samples for each accent that has more than 300 samples


accent
TENDENTE AL SICULO MA NON MARCATO    1801
BASILICATA TRENTINO                  1801
VENETO                               1393
MERIDIONALE                           134
EMILIANO                               91
Name: count, dtype: int64


# This code adjusts the audio paths and can be used for balancing the data:

In [ ]:
import pandas as pd

# add /content/accent-recog-slt2022/CommonAccent/data_small to each "wav" in the csv .../train.csv
save_dir_colab = data_path
# "content/accent-recog-slt2022/CommonAccent/data_small/"
df_train['wav'] = df_train['wav'].apply(lambda x: os.path.join(save_dir_colab, x))
df_test['wav'] = df_test['wav'].apply(lambda x: os.path.join(save_dir_colab, x))
df_dev['wav'] = df_dev['wav'].apply(lambda x: os.path.join(save_dir_colab, x))
df_train.to_csv(csv_path_train, index=False)

# show all available accents
accent_counts = df_train['accent'].value_counts()

# Print the result
print(accent_counts)

# select randomly max 300 samples for each accent
df_train2 = df_train.groupby('accent').apply(lambda x: x.sample(min(len(x), 300), random_state=1)).reset_index(drop=True)
df_dev2 = df_dev.groupby('accent').apply(lambda x: x.sample(min(len(x), 50), random_state=1)).reset_index(drop=True)
df_test2 = df_test.groupby('accent').apply(lambda x: x.sample(min(len(x), 50), random_state=1)).reset_index(drop=True)
# remove accents with less than 300 samples
df_train2 = df_train2.groupby('accent').filter(lambda x: len(x) >= 300)
# get names of remaining accents
remaining_accents = df_train2['accent'].unique()
# filter dev and test sets to keep only the remaining accents
df_dev2 = df_dev2[df_dev2['accent'].isin(remaining_accents)]
df_test2 = df_test2[df_test2['accent'].isin(remaining_accents)]
# reduce to 50 samples for dev and test sets

# print stats of the new dataframe
accent_counts2_train = df_train2['accent'].value_counts()
accent_counts2_dev = df_dev2['accent'].value_counts()
accent_counts2_test = df_test2['accent'].value_counts()
# Print the result
print(accent_counts2_train)
print(accent_counts2_dev)
print(accent_counts2_test)

# save csv again with these samples
os.makedirs('/content/accent-recog-slt2022/CommonAccent/data_small', exist_ok=True)
df_train2.to_csv('/content/accent-recog-slt2022/CommonAccent/data_small/train.csv', index=False)
df_dev2.to_csv('/content/accent-recog-slt2022/CommonAccent/data_small/dev.csv', index=False)
df_test2.to_csv('/content/accent-recog-slt2022/CommonAccent/data_small/test.csv', index=False)
# download files to disk
!zip -r /content/accent-recog-slt2022/CommonAccent/data_small.zip /content/accent-recog-slt2022/CommonAccent/data_small
# download
from google.colab import files
files.download("/content/accent-recog-slt2022/CommonAccent/data_small.zip")

accent
TENDENTE AL SICULO MA NON MARCATO    1801
BASILICATA TRENTINO                  1801
VENETO                               1393
MERIDIONALE                           134
EMILIANO                               91
Name: count, dtype: int64
accent
BASILICATA TRENTINO                  300
TENDENTE AL SICULO MA NON MARCATO    300
VENETO                               300
Name: count, dtype: int64
accent
BASILICATA TRENTINO                  50
TENDENTE AL SICULO MA NON MARCATO    50
VENETO                               50
Name: count, dtype: int64
accent
BASILICATA TRENTINO                  50
TENDENTE AL SICULO MA NON MARCATO    50
VENETO                               50
Name: count, dtype: int64


<ipython-input-20-f3c15a7b6ddf>:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_train2 = df_train.groupby('accent').apply(lambda x: x.sample(min(len(x), 300), random_state=1)).reset_index(drop=True)
<ipython-input-20-f3c15a7b6ddf>:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_dev2 = df_dev.groupby('accent').apply(lambda x: x.sample(min(len(x), 50), random_state=1)).reset_index(drop=True)
<ipytho

In [ ]:
# save audios locally (takes around 27min for italian)
for item1 in cv_19_train:
    # Check if the current file path is in the CSV file list
    # print(item['path'])
    # adjust the following line to your working directory
    filename = (item1['path'].split('/')[-2] + '/' + item1['path'].split('/')[-1])
    if filename in file_list_train or filename in file_list_test or filename in file_list_dev:
      # Decode audio
      audio_array = item1['audio']['array']  # Audio waveform
      sampling_rate = item1['audio']['sampling_rate']  # Sampling rate

      # Create the filename
      file_name = os.path.join(save_dir, filename)
      os.makedirs(os.path.dirname(file_name), exist_ok=True)

      # Save the audio as a WAV file
      write(file_name, sampling_rate, audio_array.astype(np.float32))  # Save as float32 for WAV compatibility


In [ ]:
# takes 2 min for it dev, test
for item1 in cv_19_dev:
    filename = (item1['path'].split('/')[-2] + '/' + item1['path'].split('/')[-1])
    if filename in file_list_train or filename in file_list_test or filename in file_list_dev:
      # Decode audio
      audio_array = item1['audio']['array']  # Audio waveform
      sampling_rate = item1['audio']['sampling_rate']  # Sampling rate

      # Create the filename
      file_name = os.path.join(save_dir, filename)
      os.makedirs(os.path.dirname(file_name), exist_ok=True)

      # Save the audio as a WAV file
      write(file_name, sampling_rate, audio_array.astype(np.float32))  # Save as float32 for WAV compatibility

In [ ]:
for item1 in cv_19_test:
    filename = (item1['path'].split('/')[-2] + '/' + item1['path'].split('/')[-1])
    if filename in file_list_train or filename in file_list_test or filename in file_list_dev:
      # Decode audio
      audio_array = item1['audio']['array']  # Audio waveform
      sampling_rate = item1['audio']['sampling_rate']  # Sampling rate

      # Create the filename
      file_name = os.path.join(save_dir, filename)
      os.makedirs(os.path.dirname(file_name), exist_ok=True)

      # Save the audio as a WAV file
      write(file_name, sampling_rate, audio_array.astype(np.float32))  # Save as float32 for WAV compatibility

After this step I would suggest to copy the .csv files and also the wav files to your gdrive so that you can load them from there next time.
Next time (if you saved the wav files in your gdrive) you can copy it back to colab and can continue from there.

In [ ]:
# !mkdir -p /content/accent-recog-slt2022/CommonAccent/data_small
# !cp -R /content/drive/MyDrive/.../data/wav /content/accent-recog-slt2022/CommonAccent/data_small/wav_files

In [ ]:
!cp -R /content/accent-recog-slt2022/CommonAccent/data_small/wav_files /content/drive/MyDrive/.../data/wav

In [ ]:
import soundfile as sf
import librosa
import os

# resample all files to 16khz
base_dir = '/content/accent-recog-slt2022/CommonAccent/data_small/wav_files/wav_files/'

# Walk through all subdirectories
for root, dirs, files in os.walk(base_dir):
    for filename in files:
        if filename.endswith('.mp3'):
            audio_file = os.path.join(root, filename)
            try:
                # Load with librosa to ensure mono and 16kHz resampling
                data, _ = librosa.load(audio_file, sr=16000, mono=True)
                # Output path with .wav extension
                new_path = audio_file.replace('.mp3', '.wav')
                # Save as 16kHz mono .wav
                sf.write(new_path, data, 16000)
                # print(f"Converted and saved: {new_path}")
            except Exception as e:
                print(f"Error processing {audio_file}: {e}")


# Install dependencies for training the model
You should not get any errors here

In [ ]:
!pip install -q pandas==2.2.2 \
    huggingface_hub>=0.7.0 \
    hyperpyyaml==0.0.1 \
    joblib \
    numpy==2.0.2 \
    packaging \
    pre-commit==2.3.0 \
    sentencepiece>=0.1.91 \
    SoundFile \
    tqdm

In [ ]:
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# You will probably have to adjust your audiopaths in the csv files:

In [ ]:
!cp /content/accent-recog-slt2022/CommonAccent/data_small/train.csv /content/accent-recog-slt2022/CommonAccent/data_small/train_orig.csv
!cp /content/accent-recog-slt2022/CommonAccent/data_small/dev.csv /content/accent-recog-slt2022/CommonAccent/data_small/dev_orig.csv
!cp /content/accent-recog-slt2022/CommonAccent/data_small/test.csv /content/accent-recog-slt2022/CommonAccent/data_small/test_orig.csv

In [ ]:
import os
import csv
import soundfile as sf
import librosa
import numpy as np

def update_csv_with_full_path_and_duration(
    input_csv_path,
    output_csv_path,
    base_audio_path
):
    modified_rows = []

    with open(input_csv_path, mode='r', newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)

        for row in reader:
            try:
                audio_path = os.path.join(base_audio_path, row['wav'].replace('.mp3', '.wav'))

                data, sampling_rate = sf.read(audio_path)
                duration = np.round(librosa.get_duration(y=data, sr=sampling_rate), 3)

                row['duration'] = duration
                row['wav'] = audio_path

                modified_rows.append(row)
            except Exception as e:
                print(f"Failed processing {row['wav']}: {e}")

        fieldnames = reader.fieldnames

    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(modified_rows)


In [ ]:
base_audio_path = '/content/accent-recog-slt2022/CommonAccent/data_small/wav_files/wav_files/'

# Train
update_csv_with_full_path_and_duration(
    input_csv_path='/content/accent-recog-slt2022/CommonAccent/data_small/train_orig.csv',
    output_csv_path='/content/accent-recog-slt2022/CommonAccent/data_small/train.csv',
    base_audio_path=base_audio_path
)

# Dev
update_csv_with_full_path_and_duration(
    input_csv_path='/content/accent-recog-slt2022/CommonAccent/data_small/dev_orig.csv',
    output_csv_path='/content/accent-recog-slt2022/CommonAccent/data_small/dev.csv',
    base_audio_path=base_audio_path
)

# Test
update_csv_with_full_path_and_duration(
    input_csv_path='/content/accent-recog-slt2022/CommonAccent/data_small/test_orig.csv',
    output_csv_path='/content/accent-recog-slt2022/CommonAccent/data_small/test.csv',
    base_audio_path=base_audio_path
)

In [ ]:
%cd /content/accent-recog-slt2022/CommonAccent
# make .sh file executable:
!chmod u+x run_accent_id_ecapa_tdnn.sh

/content/accent-recog-slt2022/CommonAccent


# Finally, we can run the model.
You can try to run the training using cpu by adding

--device="cpu"

to your .sh file (line 59)

In [ ]:
!./run_accent_id_ecapa_tdnn.sh

*** About to start the training ***
*** output folder: results/ECAPA-TDNN/EN/spkrec-ecapa-voxceleb/1986 ***
speechbrain.core - Beginning experiment!
speechbrain.core - Experiment folder: results/ECAPA-TDNN/EN/spkrec-ecapa-voxceleb/1986
speechbrain.dataio.encoder - Load called, but CategoricalEncoder is not empty. Loaded data will overwrite everything. This is normal if there is e.g. an unk label defined at init.
speechbrain.pretrained.fetching - Fetch embedding_model.ckpt: Delegating to Huggingface hub, source speechbrain/spkrec-ecapa-voxceleb.
speechbrain.utils.parameter_transfer - Loading pretrained files for: embedding_model
speechbrain.core - Info: device arg overridden by command line input to: cuda:0
speechbrain.core - 20.8M trainable parameters in AID
speechbrain.utils.checkpoints - Would load a checkpoint here, but none found yet.
speechbrain.utils.epoch_loop - Going into epoch 1
100% 29/29 [00:32<00:00,  1.13s/it, train_loss=7.51]
100% 5/5 [00:02<00:00,  1.77it/s]
speechbrain.

In [ ]:
# set the correct pretrained_path: /content/accent-recog-slt2022/CommonAccent/results/ECAPA-TDNN/EN/spkrec-ecapa-voxceleb/1986/save/
# and load_folder: !ref <pretrained_path>/CKPT+2025-05-15+09-43-30+00
# in the yaml file!
# if run is successful, it will create an analysis folder (in your results folder)
!python accent_id/inference.py accent_id/hparams/inference_ecapa_tdnn.yaml

speechbrain.core - Beginning experiment!
speechbrain.core - Experiment folder: results/analysis
speechbrain.dataio.encoder - Load called, but CategoricalEncoder is not empty. Loaded data will overwrite everything. This is normal if there is e.g. an unk label defined at init.
speechbrain.pretrained.fetching - Fetch embedding_model.ckpt: Using existing file/symlink in model_checkpoints/embedding_model.ckpt.
speechbrain.pretrained.fetching - Fetch classifier.ckpt: Using existing file/symlink in model_checkpoints/classifier.ckpt.
speechbrain.pretrained.fetching - Fetch accent_encoder.txt: Using existing file/symlink in model_checkpoints/label_encoder.ckpt.
speechbrain.utils.parameter_transfer - Loading pretrained files for: embedding_model, classifier, label_encoder
speechbrain.core - Info: device arg from hparam file is used
speechbrain.core - 20.8M trainable parameters in AccID_inf
100% 5/5 [00:02<00:00,  1.88it/s]
100% 5/5 [00:02<00:00,  2.47it/s]


# Our project starts here!

We have chosen the following dataset
**https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0**

for 3 languages:
*   **Russian**
*   **Belarusian**
*   **Serbian**

# Serbian

# Russian

# Belarusian

## loading the dataset

In [8]:
from google.colab import drive
drive.mount('/content/drive') #use this if you want to upload the file from colab

Mounted at /content/drive


In [9]:
gdrive_path = "/content/drive/MyDrive/Colab Notebooks/cv-corpus-14.0-delta-2023-06-23-be.tar.gz" #change to your location on Google Drive - look it up in the directory from the left
extract_path = '/content/cv_data'

import tarfile

with tarfile.open(gdrive_path, "r:gz") as tar:
    tar.extractall(path=extract_path)

print("Extraction complete.")

Extraction complete.


In [10]:
# importing from the directory
#import tarfile

#tar_path = "/content/cv-corpus-21.0-delta-2025-03-14-be.tar.gz" #your downloaded locally dataset
#extract_path = "/content/cv_data

#with tarfile.open(tar_path, "r:gz") as tar:
#    tar.extractall(path=extract_path)

#print("Extraction complete.")

## processing the dataset

In [11]:
!pip install -q speechbrain==0.5.13 \
    transformers==4.25.1 \
    librosa==0.9.2 \
    ipdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.0/499.0 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 114.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127

download_data_hf.py is a script in the loaded repo
it is situated here: /content/accent-recog-slt2022/CommonAccent/download_data_hf.py

use the code from HugginFace and the number of the dataset you need, e.g. 11
https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0


```
"""
Script to download the CommonVoice dataset using Hugging Face

At Hugging Face: https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0
Download the dataset: https://commonvoice.mozilla.org/en/datasets

def prepare_cv_from_hf(output_folder, language="en"):
    """ function to prepare the datasets in <output-folder> """

    output_folder = os.path.join(output_folder, language)
    # create the output folder: in case is not present
    os.makedirs(output_folder, exist_ok=True)

    # Prepare the the common voice dataset in streaming mode
    # common_voice_ds = load_dataset(_COMMON_VOICE_FOLDER, language, streaming=True)
    common_voice_ds = load_dataset("fsicoli/common_voice_19_0", language, streaming=True, trust_remote_code=True)


    # just select relevant splits: train/validation/test set
    splits = ["train", "validation", "test"]
    common_voice = {}
    
    # load, prepare and filter each split in streaming mode:
    for split in splits:
        # filter out samples without accent
        ds = common_voice_ds[split].filter( lambda x: x['accent'] != '')
        common_voice[split] = ds
        
    for dataset in common_voice:
        csv_lines = []
        # Starting index
        idx = 0
        for sample in common_voice[dataset]:
            # get path and utt_id
            mp3_path = sample['path']
            utt_id = mp3_path.split(".")[-2].split("/")[-1]            
            
            # Create a row with metadata + transcripts
            csv_line = [
                idx,  # ID
                utt_id,  # Utterance ID
                mp3_path,  # File name
                sample["locale"],
                sample["accent"],
                sample["age"],
                sample["gender"],
                sample["sentence"], # transcript
            ]

            # Adding this line to the csv_lines list
            csv_lines.append(csv_line)
            # Increment index
            idx += 1

        # CSV column titles
        csv_header = ["idx", "utt_id", "mp3_path", "language", "accent", "age", "gender", "transcript"]
        # Add titles to the list at indexx 0
        csv_lines.insert(0, csv_header)

        # Writing the csv lines
        csv_file = os.path.join(output_folder, dataset+'.tsv')

        with open(csv_file, mode="w", encoding="utf-8") as csv_f:
            csv_writer = csv.writer(
                csv_f, delimiter="\t", quotechar='"', quoting=csv.QUOTE_MINIMAL
            )
            for line in csv_lines:
                csv_writer.writerow(line)
    print(f"Prepare CommonVoice: for {language} in {output_folder}")

def main():
    # read input from CLI, you need to run it from the command lind
    parser = argparse.ArgumentParser()

    # reporting vars
    parser.add_argument(
        "--language",
        type=str,
        default='en',
        help="Language to load",
    )
    parser.add_argument(
        "output_folder",
        help="path of the output folder to store the csv files for each split",
    )
    args = parser.parse_args()
```



In [3]:
from datasets import load_dataset

In [5]:
common_voice_ds = load_dataset("mozilla-foundation/common_voice_11_0", "be")

n_shards.json:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

audio/be/train/be_train_0.tar:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

audio/be/train/be_train_1.tar:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

audio/be/train/be_train_2.tar:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

audio/be/train/be_train_3.tar:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

...


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 15877it [00:00, 132603.27it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 16153it [00:00, 110927.07it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 27726it [00:00, 140040.45it/s]


In [ ]:
!python /content/accent-recog-slt2022/CommonAccent/download_data_hf.py --language "be" /content/cv_data

Reading metadata...: 347670it [00:10, 31839.28it/s]


common_accent_prepare.py is a script in the loaded repo
it is situated here: /content/accent-recog-slt2022/CommonAccent/common_accent_prepare.py

```
def prepare_common_accent(
        data_folder,
        save_folder,
        accented_letters=False,
        language="en",        
        skip_prep=False,
    ):

    """
    Prepares the csv files for the CommonAccent dataset for Accent Classification.
    Download: https://commonvoice.mozilla.org/en/datasets

    Arguments
    ---------
    data_folder : str
        Path to the folder where the CommonAccent dataset for Accent Classification is stored.
        This path should include the multi: /datasets/CommonAccent
    save_folder : str
        The directory where to store the csv files.
    max_duration : int, optional
        Max duration (in seconds) of training uterances.
    accented_letters : bool, optional
        Defines if accented letters will be kept as individual letters or
        transformed to the closest non-accented letters.
    language: str
        Specify the language for text normalization.        
    skip_prep: bool
        If True, skip data preparation.

    Example
    -------
    >>> from recipes.CommonAccent.common_accent_prepare import prepare_common_accent
    >>> data_folder = '/datasets/CommonAccent'
    >>> save_folder = 'exp/CommonAccent_exp'
    >>> prepare_common_accent(\
            data_folder,\
            save_folder,\
            skip_prep=False\
        )
    """
```


In [ ]:
import sys
sys.path.append("/content/accent-recog-slt2022")

from CommonAccent.common_accent_prepare import prepare_common_accent

prepare_common_accent(
    data_folder="/content/cv_data/be",
    save_folder="/content/common_accent_output",
    language="be",
    skip_prep=False
)

In [19]:
!python common_accent_prepare.py --language "be" /content/cv_data /content/common_accent_output

Traceback (most recent call last):
  File "/content/accent-recog-slt2022/CommonAccent/common_accent_prepare.py", line 570, in <module>
    main()
  File "/content/accent-recog-slt2022/CommonAccent/common_accent_prepare.py", line 464, in main
    prepare_common_accent(args.cv_folder, args.output_folder, language=args.language)
  File "/content/accent-recog-slt2022/CommonAccent/common_accent_prepare.py", line 169, in prepare_common_accent
    check_common_accent_folder(data_folder, language=language)
  File "/content/accent-recog-slt2022/CommonAccent/common_accent_prepare.py", line 440, in check_common_accent_folder
    raise FileNotFoundError(err_msg)
FileNotFoundError: /content/cv_data/be must have at least 'train.tsv' file in it.


In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.io.wavfile import write
from datasets import load_dataset

data_path = "/content/accent-recog-slt2022/CommonAccent/data/"

# Load CSV
csv_path_train = data_path + "train.csv"
df_train = pd.read_csv(csv_path_train)
file_list_train = list(set(df_train['wav']))

csv_path_test = data_path + "test.csv"
df_test = pd.read_csv(csv_path_test)
file_list_test = list(set(df_test['wav']))

csv_path_dev = data_path + "dev.csv"
df_dev = pd.read_csv(csv_path_dev)
file_list_dev = list(set(df_dev['wav']))

# Set save directory
save_dir = data_path + "wav_files"
os.makedirs(save_dir, exist_ok=True)

# Load dataset in streaming mode
language = "be"  # or whatever language code you're using
cv_11_be_train = load_dataset("mozilla-foundation/common_voice_11_0", language, streaming=True, split="train", trust_remote_code=True)
print(next(iter(cv_11_be_train)))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Reading metadata...: 171388it [00:07, 23662.65it/s]


{'client_id': 'c14f21cacf2b7701ad0bead0dd1b31ec9d3a1557708e446de55e98b4b470cf31072c82543e5ba518c5c187a91868878a4e32727054a3dd94f9df41c9a13d8c62', 'path': 'it_train_0/common_voice_it_17415777.mp3', 'audio': {'path': 'it_train_0/common_voice_it_17415777.mp3', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
       -1.61342832e-05, -1.57291943e-05, -1.16234423e-05]), 'sampling_rate': 48000}, 'sentence': "Il marchese aveva già moglie in quell'epoca?", 'up_votes': 3, 'down_votes': 0, 'age': '', 'gender': '', 'accent': '', 'locale': 'it', 'segment': '', 'variant': ''}


In [ ]:
cv_11_be_dev = load_dataset("mozilla-foundation/common_voice_11_0", language, streaming=False, split="validation", trust_remote_code=True)
cv_11_be_test = load_dataset("mozilla-foundation/common_voice_11_0", language, streaming=False, split="test", trust_remote_code=True)

In [ ]:
import pandas as pd
# Count entries per accent
accent_counts = df_train['accent'].value_counts()

# Print the result
print(accent_counts)

# select randomly max 300 samples for each accent that has more than 300 samples

accent
TENDENTE AL SICULO MA NON MARCATO    1801
BASILICATA TRENTINO                  1801
VENETO                               1393
MERIDIONALE                           134
EMILIANO                               91
Name: count, dtype: int64


## balancing

In [ ]:
import pandas as pd

# add /content/accent-recog-slt2022/CommonAccent/data_small to each "wav" in the csv .../train.csv
save_dir_colab = data_path
# "content/accent-recog-slt2022/CommonAccent/data_small/"
df_train['wav'] = df_train['wav'].apply(lambda x: os.path.join(save_dir_colab, x))
df_test['wav'] = df_test['wav'].apply(lambda x: os.path.join(save_dir_colab, x))
df_dev['wav'] = df_dev['wav'].apply(lambda x: os.path.join(save_dir_colab, x))
df_train.to_csv(csv_path_train, index=False)

# show all available accents
accent_counts = df_train['accent'].value_counts()

# Print the result
print(accent_counts)

# select randomly max 300 samples for each accent
df_train2 = df_train.groupby('accent').apply(lambda x: x.sample(min(len(x), 300), random_state=1)).reset_index(drop=True)
df_dev2 = df_dev.groupby('accent').apply(lambda x: x.sample(min(len(x), 50), random_state=1)).reset_index(drop=True)
df_test2 = df_test.groupby('accent').apply(lambda x: x.sample(min(len(x), 50), random_state=1)).reset_index(drop=True)
# remove accents with less than 300 samples
df_train2 = df_train2.groupby('accent').filter(lambda x: len(x) >= 300)
# get names of remaining accents
remaining_accents = df_train2['accent'].unique()
# filter dev and test sets to keep only the remaining accents
df_dev2 = df_dev2[df_dev2['accent'].isin(remaining_accents)]
df_test2 = df_test2[df_test2['accent'].isin(remaining_accents)]
# reduce to 50 samples for dev and test sets

# print stats of the new dataframe
accent_counts2_train = df_train2['accent'].value_counts()
accent_counts2_dev = df_dev2['accent'].value_counts()
accent_counts2_test = df_test2['accent'].value_counts()
# Print the result
print(accent_counts2_train)
print(accent_counts2_dev)
print(accent_counts2_test)

# save csv again with these samples
os.makedirs('/content/accent-recog-slt2022/CommonAccent/data_small', exist_ok=True)
df_train2.to_csv('/content/accent-recog-slt2022/CommonAccent/data_small/train.csv', index=False)
df_dev2.to_csv('/content/accent-recog-slt2022/CommonAccent/data_small/dev.csv', index=False)
df_test2.to_csv('/content/accent-recog-slt2022/CommonAccent/data_small/test.csv', index=False)
# download files to disk
!zip -r /content/accent-recog-slt2022/CommonAccent/data_small.zip /content/accent-recog-slt2022/CommonAccent/data_small
# download
from google.colab import files
files.download("/content/accent-recog-slt2022/CommonAccent/data_small.zip")

In [ ]:
import pandas as pd
import os

# Define paths
data_path = "..."  # where the .wav files are
csv_path_train = "/content/common_accent_output/train.csv"
csv_path_dev = "/content/common_accent_output/dev.csv"
csv_path_test = "/content/common_accent_output/test.csv"

# Load CSVs
df_train = pd.read_csv(csv_path_train, sep="\t")
df_dev = pd.read_csv(csv_path_dev, sep="\t")
df_test = pd.read_csv(csv_path_test, sep="\t")

FileNotFoundError: [Errno 2] No such file or directory: '/content/common_accent_output/train.csv'

In [ ]:
# Add full path to each 'wav' file (clip name)
df_train['wav'] = df_train['wav'].apply(lambda x: os.path.join(data_path, x))
df_dev['wav'] = df_dev['wav'].apply(lambda x: os.path.join(data_path, x))
df_test['wav'] = df_test['wav'].apply(lambda x: os.path.join(data_path, x))

In [ ]:
# Show all accents
print(df_train['accent'].value_counts())

# Sample & filter
df_train2 = df_train.groupby('accent').apply(lambda x: x.sample(min(len(x), 300), random_state=1)).reset_index(drop=True)
df_train2 = df_train2.groupby('accent').filter(lambda x: len(x) >= 300)

remaining_accents = df_train2['accent'].unique()
df_dev2 = df_dev[df_dev['accent'].isin(remaining_accents)].groupby('accent').apply(lambda x: x.sample(min(len(x), 50), random_state=1)).reset_index(drop=True)
df_test2 = df_test[df_test['accent'].isin(remaining_accents)].groupby('accent').apply(lambda x: x.sample(min(len(x), 50), random_state=1)).reset_index(drop=True)


In [ ]:
# Save new CSVs
save_path = "/content/accent-recog-slt2022/CommonAccent/data_small"
os.makedirs(save_path, exist_ok=True)

df_train2.to_csv(os.path.join(save_path, "train.csv"), index=False)
df_dev2.to_csv(os.path.join(save_path, "dev.csv"), index=False)
df_test2.to_csv(os.path.join(save_path, "test.csv"), index=False)

# Zip and download
!zip -r /content/data_small.zip {save_path}
from google.colab import files
files.download("/content/data_small.zip")

In [ ]:
# save audios locally

for item1 in cv_11_be_train:
    # Check if the current file path is in the CSV file list
    # print(item['path'])
    # adjust the following line to your working directory
    filename = (item1['path'].split('/')[-2] + '/' + item1['path'].split('/')[-1])
    if filename in file_list_train or filename in file_list_test or filename in file_list_dev:
      # Decode audio
      audio_array = item1['audio']['array']  # Audio waveform
      sampling_rate = item1['audio']['sampling_rate']  # Sampling rate

      # Create the filename
      file_name = os.path.join(save_dir, filename)
      os.makedirs(os.path.dirname(file_name), exist_ok=True)

      # Save the audio as a WAV file
      write(file_name, sampling_rate, audio_array.astype(np.float32))  # Save as float32 for WAV compatibility

In [ ]:
# takes 2 min for it dev, test
for item1 in cv_11_be_train:
    filename = (item1['path'].split('/')[-2] + '/' + item1['path'].split('/')[-1])
    if filename in file_list_train or filename in file_list_test or filename in file_list_dev:
      # Decode audio
      audio_array = item1['audio']['array']  # Audio waveform
      sampling_rate = item1['audio']['sampling_rate']  # Sampling rate

      # Create the filename
      file_name = os.path.join(save_dir, filename)
      os.makedirs(os.path.dirname(file_name), exist_ok=True)

      # Save the audio as a WAV file
      write(file_name, sampling_rate, audio_array.astype(np.float32))  # Save as float32 for WAV compatibility

In [ ]:
for item1 in cv_11_be_train:
    filename = (item1['path'].split('/')[-2] + '/' + item1['path'].split('/')[-1])
    if filename in file_list_train or filename in file_list_test or filename in file_list_dev:
      # Decode audio
      audio_array = item1['audio']['array']  # Audio waveform
      sampling_rate = item1['audio']['sampling_rate']  # Sampling rate

      # Create the filename
      file_name = os.path.join(save_dir, filename)
      os.makedirs(os.path.dirname(file_name), exist_ok=True)

      # Save the audio as a WAV file
      write(file_name, sampling_rate, audio_array.astype(np.float32))  # Save as float32 for WAV compatibility

## training

In [ ]:
!./run_accent_id_ecapa_tdnn.sh

*** About to start the training ***
*** output folder: results/ECAPA-TDNN/EN/spkrec-ecapa-voxceleb/1986 ***
speechbrain.core - Beginning experiment!
speechbrain.core - Experiment folder: results/ECAPA-TDNN/EN/spkrec-ecapa-voxceleb/1986
speechbrain.dataio.encoder - Load called, but CategoricalEncoder is not empty. Loaded data will overwrite everything. This is normal if there is e.g. an unk label defined at init.
speechbrain.pretrained.fetching - Fetch embedding_model.ckpt: Delegating to Huggingface hub, source speechbrain/spkrec-ecapa-voxceleb.
speechbrain.utils.parameter_transfer - Loading pretrained files for: embedding_model
speechbrain.core - Info: device arg overridden by command line input to: cuda:0
speechbrain.core - 20.8M trainable parameters in AID
speechbrain.utils.checkpoints - Would load a checkpoint here, but none found yet.
speechbrain.utils.epoch_loop - Going into epoch 1
100% 29/29 [00:32<00:00,  1.13s/it, train_loss=7.51]
100% 5/5 [00:02<00:00,  1.77it/s]
speechbrain.

In [ ]:
# set the correct pretrained_path: /content/accent-recog-slt2022/CommonAccent/results/ECAPA-TDNN/EN/spkrec-ecapa-voxceleb/1986/save/
# and load_folder: !ref <pretrained_path>/CKPT+2025-05-15+09-43-30+00
# in the yaml file!
# if run is successful, it will create an analysis folder (in your results folder)
!python accent_id/inference.py accent_id/hparams/inference_ecapa_tdnn.yaml

speechbrain.core - Beginning experiment!
speechbrain.core - Experiment folder: results/analysis
speechbrain.dataio.encoder - Load called, but CategoricalEncoder is not empty. Loaded data will overwrite everything. This is normal if there is e.g. an unk label defined at init.
speechbrain.pretrained.fetching - Fetch embedding_model.ckpt: Using existing file/symlink in model_checkpoints/embedding_model.ckpt.
speechbrain.pretrained.fetching - Fetch classifier.ckpt: Using existing file/symlink in model_checkpoints/classifier.ckpt.
speechbrain.pretrained.fetching - Fetch accent_encoder.txt: Using existing file/symlink in model_checkpoints/label_encoder.ckpt.
speechbrain.utils.parameter_transfer - Loading pretrained files for: embedding_model, classifier, label_encoder
speechbrain.core - Info: device arg from hparam file is used
speechbrain.core - 20.8M trainable parameters in AccID_inf
100% 5/5 [00:02<00:00,  1.88it/s]
100% 5/5 [00:02<00:00,  2.47it/s]


## evaluation with confusion matrix, error rate, precision, and F1-score

## visualisation wit t-sne